In [127]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=RuntimeWarning)

import sys
import logging

from scipy.io import arff

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skmultiflow.data import DataStream
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr
from util import plot_performance_comparison, print_evaluation, plot_drift_detection_summary
import pickle

%load_ext autoreload
%autoreload 2

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [134]:
all_results = {}

key = 'Air_Quality'
local = {}
filename = '../Experiment_AirQuality/results_MCDropout_air.pickle'
local['raw'] = pickle.load(open(filename, 'rb'))
all_results[key] = local

key = 'Bike_Sharing'
local = {}
filename = '../Experiment_BikeSharing/results_MCDropout_bike.pickle'
local['raw'] = pickle.load(open(filename, 'rb'))
all_results[key] = local

# Synthetic

key = 'Syn_Friedman'
local = {}
filename = '../Syn_Experiment_Friedman_Abrupt/results_MCDropout_friedman.pickle'
local['raw'] = pickle.load(open(filename, 'rb'))
all_results[key] = local

In [135]:
for key in all_results.keys():
    all_results[key]['summary'] = print_evaluation(all_results[key]['raw'])

In [136]:
for key in all_results.keys():
    table = all_results[key]['summary']
    table.insert(1, 'RMSE', np.sqrt(table['MSE']))
    all_results[key]['summary'] = table

#### Metric

In [137]:
all_results[key]['summary'].keys()

Index(['MAE', 'RMSE', 'MSE', 'SMAPE', 'r_spear', 'r_pears',
       'detected_drift_numbers', 'retraining_counter', 'Computation_Time'],
      dtype='object')

In [140]:
metric = 'SMAPE'
overview = {}

for key in all_results.keys():
    overview[key] = all_results[key]['summary'][metric]

In [141]:
print(metric)
results = pd.DataFrame(overview).T
ordered_columnns = ['no_retraining', 'uninformed', 'equal_distribution', 'kswin', 'adwin_uncertainty','kswin_unlimited',  'adwin_error']
results[ordered_columnns]

SMAPE


Detection,no_retraining,uninformed,equal_distribution,kswin,adwin_uncertainty,kswin_unlimited,adwin_error
Air_Quality,12.464060,10.136977,8.843035,10.556120,8.963886,10.332211,10.678390
Bike_Sharing,71.882171,62.756937,56.938133,61.037776,55.487333,46.666323,54.741980
Syn_Friedman,25.647768,21.263897,22.821049,23.083311,17.227913,15.099723,16.664773


#### Number retraining

In [142]:
metric = 'retraining_counter'
overview = {}

for key in all_results.keys():
    overview[key] = all_results[key]['summary'][metric]

In [143]:
print(metric)
results = pd.DataFrame(overview).T
ordered_columnns = ['no_retraining', 'uninformed', 'equal_distribution', 'kswin', 'adwin_uncertainty','kswin_unlimited',  'adwin_error']
results[ordered_columnns]

retraining_counter


Detection,no_retraining,uninformed,equal_distribution,kswin,adwin_uncertainty,kswin_unlimited,adwin_error
Air_Quality,0,14,14,14,14,19,12
Bike_Sharing,0,5,5,5,5,27,8
Syn_Friedman,0,3,3,3,3,20,5


#### Details

In [32]:
all_results['Air_Quality']['summary']

,MAE,RMSE,MSE,SMAPE,r_spear,r_pears,detected_drift_numbers,retraining_counter,Computation_Time
Detection,,,,,,,,,
adwin_error,0.847367,1.386682,1.922888,10.678390,0.445291,0.618342,12,12,93.672636
adwin_uncertainty,0.687711,1.151485,1.325917,8.963886,0.424870,0.411051,14,14,109.424100
equal_distribution,0.744370,1.230825,1.514930,8.843035,0.568438,0.495504,0,14,101.274468
kswin,0.821992,1.307496,1.709546,10.936935,0.372710,0.631231,12,12,116.628383
kswin_unlimited,0.801440,1.266734,1.604614,10.824041,0.373594,0.626503,14,14,130.606989
no_retraining,0.766381,1.169737,1.368285,12.464060,0.015409,0.516991,0,0,11.691438
uninformed,0.880579,1.439194,2.071278,10.087713,0.597060,0.629400,0,14,105.213869


In [8]:
all_results['Bike_Sharing']['summary']

,MAE,RMSE,MSE,SMAPE,r_spear,r_pears,detected_drift_numbers,retraining_counter,Computation_Time
Detection,,,,,,,,,
adwin_error,108.322724,160.733777,25835.347153,67.211094,0.279784,0.030117,6,6,54.846816
adwin_uncertainty,100.053476,148.359027,22010.401004,63.216678,0.366978,0.161331,7,7,49.729426
equal_distribution,106.990447,153.537804,23573.857177,67.286410,0.374176,0.137045,0,7,50.453768
kswin,106.542359,156.858752,24604.668083,65.906629,0.309038,0.024824,7,7,105.342770
kswin_unlimited,98.438282,149.007880,22203.348224,61.696364,0.408112,0.131316,28,27,168.490429
no_retraining,170.958287,210.939111,44495.308578,82.682654,0.451827,0.305720,0,0,11.289326
uninformed,116.668695,165.015595,27230.146699,69.638265,0.373971,0.142420,0,7,52.442428


In [18]:
bike = all_results['Bike_Sharing']['raw']

In [19]:
bike['raw'].keys()

dict_keys(['adwin_uncertainty 0', 'uninformed_1', 'uninformed_2', 'uninformed_3', 'uninformed_4', 'uninformed_5', 'no_retraining', 'equal_distribution', 'adwin_error', 'kswin', 'kswin_unlimited'])

In [20]:
bike['raw']['adwin_uncertainty 0'].keys()

dict_keys(['uncertainties', 'tv', 'tv_cat', 'preds', 'probs', 'errors'])

In [21]:
predictions = bike['raw']['adwin_uncertainty 0']['preds']
true_values = bike['raw']['adwin_uncertainty 0']['tv']

In [22]:
np.sqrt(mean_squared_error(true_values, predictions))

148.35902737586878

In [23]:
uncertainties = bike['raw']['adwin_uncertainty 0']['uncertainties']
errors = bike['raw']['adwin_uncertainty 0']['errors']

In [24]:
spearmanr(errors, uncertainties)[0]

0.36697834919551975

In [25]:
np.corrcoef(errors, uncertainties)[0, 1]

0.16133101904315064

In [101]:
for data_set in all_results.keys():
    df_prel = all_results[data_set]['raw']['metrics']
    print(data_set,': ', df_prel[df_prel.Detection == 'adwin_uncertainty']['r_spear'][0])

Air_Quality :  0.42487026152727925
Bike_Sharing :  0.36697834919551975
Syn_Friedman :  0.473779234047615
